# importing the neccessary libraries to collab
import pandas as pd
import numpy as np
import smtplib as sm
!pip install XlsxWriter
import xlsxwriter



In [ ]:
import pandas as pd
import numpy as np
!pip install XlsxWriter
import xlsxwriter
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
final_datasets=['American Modern Ins Grp Inc',
 'Amerisafe Grp',
 'Overseas Partners Us Reins Co',
 'Hyundai Marine & Fire Ins Co Ltd',
 'Martingale Natl Ins Co',
 'Proformance Ins Co (The)',
 'Catholic Relief Ins Co Of Amer',
 'First Amer Ins Co',
 'Generali Us Branch',
 'Underwriters At Lloyds London',
 'Cooperative Mut Ins Co',
 'Elevators Mut Ins Co',
 'Mennonite Mut Ins Co',
 'Co-Operative Ins Co',
 'New York Municipal Ins Reciprocal',
 'Housing & Redevelopment Ins Exch',
 'Middle States Ins Co Inc',
 'Contractors Bonding & Ins Co',
 'Pacific Specialty Ins Co',
 'Omega Ins Co',
 'Southern Group Ind Inc',
 'Nova Cas Co',
 'Atlantic Cas Ins Co',
 'Dowa Fire & Marine Ins Co Ltd Us Br',
 'Yel Co Ins']



In [ ]:
len(final_datasets)

25

In [ ]:
publish='/content/drive/MyDrive/Colab resources/CsOutstandingProject/Results_by_normal_method/'

In [ ]:
len(final_datasets)

26

In [ ]:
class CaseOutstanding_Reserving:
  def __init__(self,df,accident_year,reported_year,reported_loss,paid_reported_loss,case_outstanding):
    self.df=df
    self.accident_year=accident_year
    self.reported_year=reported_year
    self.reported_loss=reported_loss
    self.paid_reported_loss=paid_reported_loss
    self.case_outstanding=case_outstanding


# This method will help you to build triangle given the parameter
  def d_triangle(self,parameter):
      columns=[]
      max=0
      for i in self.df[self.accident_year].unique():
        a=self.df.groupby(self.accident_year)
        a=a.get_group(i)
        l=len(a)
        if max<l:
          max=l
      final=[]
      for i in range(1,max+1):
        columns.append(12*i)
      j=0
      for i in self.df[self.accident_year].unique():
        check=1
        lis=[]
        while check and j<len(self.df):
          if j+1 <= len(self.df):
            lis.append(self.df.loc[j,parameter])
          if j+1 >= len(self.df):
            check=0
          else:
            curr=self.df[self.accident_year][j]
            next=self.df[self.accident_year][j+1]
          if next != curr:
            check=0
          j=j+1
        if (len(lis)<max):
          for i in range(0,max-len(lis)):
            lis.append(np.NaN)
        final.append(lis)
      bf=pd.DataFrame(final,columns=columns)
      bf["AccidentYear"]=self.df[self.accident_year].unique()
      first_column = bf.pop('AccidentYear')
      bf.insert(0,"AccidentYear",first_column)
      return bf


#This method will help in building incremental triangles especially if you wanna convert cummulative paid claims in to incremental ones
  def incremental_triangles(self):
    final=[]
    for i in range(0,len(self.d_triangle(self.paid_reported_loss))):
      val=[]
      for j in range(0,len(self.d_triangle(self.paid_reported_loss).columns)):
        if j==0 or j==1:
          g=self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]
        else:
          if self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]!=np.NaN:
            g=self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]-self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j-1]]
          else:
            g=np.NaN
        val.append(g)
      final.append(val)
    inc=pd.DataFrame(final,columns=self.d_triangle(self.paid_reported_loss).columns)
    return inc


#This will help you to pai
  def paid_tr_on_case_ratio(self):
    final=[]
    li=list(self.d_triangle(self.paid_reported_loss).columns[1:])
    bi=list(self.d_triangle(self.paid_reported_loss).columns[2:])
    for i in range(0,len(self.d_triangle(self.paid_reported_loss))-1):
      val=[]
      for j in range(0,len(li)-1):
        if self.d_triangle(self.paid_reported_loss).loc[i].at[li[j+1]]!=np.NaN and self.d_triangle(self.paid_reported_loss).loc[i].at[li[j]]!=np.NaN:
          g=self.d_triangle(self.paid_reported_loss).loc[i].at[li[j+1]]-self.d_triangle(self.paid_reported_loss).loc[i].at[li[j]]
        else:
          g=np.NaN
        if self.d_triangle(self.case_outstanding).loc[i].at[li[j]]!=np.NaN:
          f=self.d_triangle(self.case_outstanding).loc[i].at[li[j]]
        else:
          f=np.NaN
        if f!=np.NaN and g!=np.NaN:
          val.append(g/f)
        else:
          val.append(np.NaN)
      final.append(val)
    nf=pd.DataFrame(final,columns=bi)
    nf[self.accident_year]=self.d_triangle(self.paid_reported_loss)[self.accident_year]
    first_column = nf.pop(self.accident_year)
    nf.insert(0,self.accident_year,first_column)
    return nf


  def rem_in_case_ratio(self):
    li=list(self.d_triangle(self.case_outstanding).columns[1:])
    bi=li[1:]
    final=[]
    for i in range(0,len(self.d_triangle(self.case_outstanding))-1):
      val=[]
      for j in range(0,len(li)-1):
        if self.d_triangle(self.case_outstanding).loc[i].at[li[j]]!=np.NaN and self.d_triangle(self.case_outstanding).loc[i].at[li[j+1]]!=np.NaN:
          c=self.d_triangle(self.case_outstanding).loc[i].at[li[j+1]]/self.d_triangle(self.case_outstanding).loc[i].at[li[j]]
          val.append(c)
        else:
          val.append(np.NaN)
      final.append(val)
    nf=pd.DataFrame(final,columns=bi)
    nf=pd.DataFrame(final,columns=bi)
    nf[self.accident_year]=self.d_triangle(self.case_outstanding)[self.accident_year]
    first_column = nf.pop(self.accident_year)
    nf.insert(0,self.accident_year,first_column)
    return nf



  def calc_measures(self,ratio_triangle):
    columns=list(ratio_triangle.columns)
    final=[]
    for i in columns[1:]:
      val=[]
      mean=ratio_triangle[i].mean()
      median=ratio_triangle[i].median()
      val.append(mean)
      val.append(median)
      final.append(val)
    bf=pd.DataFrame(final)
    bf.columns=["Mean","Median"]
    bf["Months"]=(bf.index+2)*12
    bf.transpose()
    return bf

  def fill_triangles_case_outstanding(self,measure,method):
    dup=self.d_triangle(self.case_outstanding)
    columns=list(dup.columns)
    for i in range(0,len(dup)):
      for j in range(1,len(columns)):
        if dup.isnull().iloc[i,j]:
          dup.loc[i,columns[j]]=round(measure[method][j-2]*dup.loc[i].at[columns[j-1]],3)
    return dup

  def fill_triangles_paid_triangle(self,case_out,paid_inc,measure,method):
    columns=list(case_out.columns)
    for i in range(0,len(case_out)):
      for j in range(1,len(columns)):
        if paid_inc.isnull().loc[i,columns[j]]:
          paid_inc.loc[i,columns[j]]=(measure[method][j-2]*case_out.loc[i].at[columns[j-1]])
    return paid_inc











In [ ]:
a=[]

for i in final_datasets:

  li=[]
  name=i
  print("The name of the dataset is:",i)
  initial=pd.read_csv('/content/drive/MyDrive/Colab resources/CsOutstandingProject/gen_for_collab/'+i+'.csv')
  writer=pd.ExcelWriter(publish+i+".xlsx",engine='xlsxwriter')
  initial.to_excel(writer,sheet_name="Raw")
  final=pd.read_csv('/content/drive/MyDrive/Colab resources/CsOutstandingProject/Gen_by_code not final/ok_data_sets/'+i+'.csv')

  obj=CaseOutstanding_Reserving(initial,"AccidentYear","DevelopmentYear","Cummulative_loss","Cummulative_paid_loss","Cummulative_case_outstanding")
  obj1=CaseOutstanding_Reserving(final,"AccidentYear","DevelopmentYear","Cummulative_loss","Cummulative_paid_loss","Cummulative_case_outstanding")



  parameter= 'Cummulative_paid_loss'
  final=obj1.d_triangle(parameter)
  final.to_excel(writer,sheet_name="final")
  print("the final dataset looks like this:")
  print(final)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")


  #trying to get the case outstanding triangle
  parameter="Cummulative_case_outstanding"
  case_out_triangle=obj.d_triangle(parameter)
  case_out_triangle.to_excel(writer,sheet_name="case_out_triangle")
  print("\n\nThis is the case_outstanding Triangle")
  print(case_out_triangle)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  #getting the paid triangle
  parameter="Cummulative_paid_loss"
  paid_triangle=obj.d_triangle(parameter)


  #building the incremental paid triangles from the cummulative ones above
  paid_incremental_triangle=obj.incremental_triangles()
  paid_incremental_triangle.to_excel(writer,sheet_name="paid_incremental")
  print("\n\nThis is the Incremental Paid Triangle")
  print(paid_incremental_triangle)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  #building the paid ratio triangle
  paid_ratio_triangle=obj.paid_tr_on_case_ratio()
  paid_ratio_triangle.to_excel(writer,sheet_name="paid_ratio_triangle")
  print("\n\nThis is the paid ratio Triangle")
  print(paid_ratio_triangle)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  plt.subplots(figsize=(15,8))
  dg=sns.heatmap(data=paid_ratio_triangle.drop("AccidentYear",axis=1),annot=True,cmap=sns.color_palette("viridis_r", as_cmap=True),fmt=".00005f",yticklabels=paid_ratio_triangle["AccidentYear"])
  plt.title("Paid_Incremental_ratios")
  plt.show()

  #building the case ratio triangle
  case_ratio_triangle=obj.rem_in_case_ratio()
  case_ratio_triangle.to_excel(writer,sheet_name="case_ratio_triangle")
  print("\n\nThis is the case_ratio_Triangle")
  print(case_ratio_triangle)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  plt.subplots(figsize=(15,8))
  some=sns.heatmap(data=case_ratio_triangle.drop("AccidentYear",axis=1),annot=True,cmap=sns.color_palette("viridis_r", as_cmap=True),fmt=".00005f",yticklabels=case_ratio_triangle["AccidentYear"])
  plt.title("Remaining_in_case_ratios")
  plt.show()

  #getting the averages for case_outstanding triangle
  measures_case_out=obj.calc_measures(case_ratio_triangle)
  measures_case_out.to_excel(writer,sheet_name="case_averages")
  print("\n\nThis is the case_out measures or averages Triangle")
  print(measures_case_out)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  measures_case_out.set_index("Months").plot(kind="line")
  plt.title("remaining_in_case_age_to_age_factors")
  plt.show()

  #getting the averages for incremental paid triangle
  measures_paid_tr=obj.calc_measures(paid_ratio_triangle)
  measures_paid_tr.to_excel(writer,sheet_name="paid_averages")
  print("\n\nThis is the paid incremental measures or averages Triangle")
  print(measures_paid_tr)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  measures_paid_tr.set_index("Months").plot(kind="line")
  plt.title("Inc_paid_age to age_ratios")
  plt.show()

  #filled the case_outstanding triangle
  nf=case_out=obj.fill_triangles_case_outstanding(measures_case_out,"Mean")
  nf.to_excel(writer,sheet_name="filled_case_reserves")
  print("\n\nThis is the filled case_outstanding Triangle")
  print(nf)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

  nf.set_index(nf["AccidentYear"])[nf.columns[-1]].plot(kind="bar",figsize=(15,10),color="Red")
  plt.title("Case_outstanding_claims")
  plt.show()

  #filled the paid_triangles
  mf=obj.fill_triangles_paid_triangle(case_out,paid_incremental_triangle,measures_paid_tr,"Mean")

  #cummulatig the paid triangles to get the final cost
  for i in range(2,len(mf.columns)):
    mf[mf.columns[i]]=mf[mf.columns[i]]+mf[mf.columns[i-1]]

  mf.to_excel(writer,sheet_name="filled_paid_triangles")
  print("\n\nThis is the cummulative filled incremental paid claims Triangle")
  print(mf)
  mf.set_index(mf["AccidentYear"])[mf.columns[-1]].plot(kind="bar",figsize=(15,10),color="Orange")
  plt.title("Cummulative_incremental_Claims")
  plt.show()
  writer.save()
  #comparing the results
  last=mf.columns[-1]
  print(final[last].sum())
  print(mf[last].sum())
  li.append(name)
  li.append(final[last].sum())
  li.append(mf[last].sum())
  a.append(li)
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")
  print("----------------------------------------------------------------------------------------------------------------------------------------------------")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
results=pd.DataFrame(a,columns=["Name","original","predicted"])

In [ ]:
results

,Name,original,predicted
0,American Modern Ins Grp Inc,8038.0,11027.784343
1,Amerisafe Grp,212.0,211.083913
2,Overseas Partners Us Reins Co,2001.0,2302.116949
3,Hyundai Marine & Fire Ins Co Ltd,28.0,16.150480
4,Martingale Natl Ins Co,6122.0,6443.036435
5,Proformance Ins Co (The),3209.0,2821.980348
6,Catholic Relief Ins Co Of Amer,787.0,787.000000
7,First Amer Ins Co,10019.0,12310.670410
8,Generali Us Branch,15069.0,12598.378551
9,Underwriters At Lloyds London,312.0,319.325027


In [ ]:
results["error"]=(results["original"]-results["predicted"])
results["error_wrt_original"]=(results["error"]/results["original"])*100

In [ ]:
writer=pd.ExcelWriter('/content/drive/MyDrive/Colab resources/CsOutstandingProject/initial_reports_for_ok_data_sets'+".xlsx",engine='xlsxwriter')
results.to_excel(writer,sheet_name="Results for good_data_sets")
writer.save()

<ipython-input-27-4be87e1138cf>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:
sk=StandardScaler()
v1=sk.fit_transform(results[["original","predicted"]])


In [ ]:
ch=pd.DataFrame(v1,columns=["original","predicted"])

In [34]:
results[(results["error_wrt_original"] < 10) & (results["error_wrt_original"] > -10)]["error_wrt_original"].count()

9

In [35]:
results[(results["error_wrt_original"] < 1) & (results["error_wrt_original"] > -1)]["error_wrt_original"].count()

4

In [ ]:
mae=mean_absolute_error(ch['original'],ch['predicted'])

In [ ]:
mse=mean_squared_error(ch['original'],ch['predicted'])

In [ ]:
mae

0.12462294193949244

In [ ]:
mse

0.06449797915890235